In [1]:
!pip install flask
!pip install flask-ngrok
!pip install pyngrok
!ngrok authtoken xxx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.2/732.2 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-7.0.2-py3-none-any.whl size=21233 sha256=07ac99ca9ee9e76a34d5fd640be91a5d20afef93cebbee0c8ffe23042b8461bc
  Stored in directory: /root/.cache/pip/wheels/63/3f/ca/3ee63fa3bf9dfcf6014bb5ea56026c8b218ad26b422fd9cce1
Successfully built pyngrok
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [2]:
from flask import Flask, render_template, request, url_for, flash, redirect, abort
import sqlite3
import os
import threading
import json,urllib.request
from pyngrok import ngrok

app = Flask(__name__)
port = "5000"


public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))


app.config["BASE_URL"] = public_url

app = Flask(__name__)
app.secret_key = 'super secret key'
app.config['SESSION_TYPE'] = 'filesystem'

def get_db_connection():
    conn = sqlite3.connect('test.db')
    conn.row_factory = sqlite3.Row
    return conn


@app.route('/')
def index():
    conn = get_db_connection()
    posts = conn.execute('SELECT * FROM COMPANY').fetchall()
    conn.close()
    return render_template('index.html', posts=posts)

@app.route('/create', methods=('GET', 'POST'))
def create():
    if request.method == 'POST':
        ID = request.form['ID']
        NAME = request.form['NAME']
        AGE = request.form['AGE']
        ADDRESS = request.form['ADDRESS']
        SALARY = request.form['SALARY']

        if not ID:
            flash('Title is required!')
        elif not NAME :
            flash('Content is required!')
        else:
            conn = get_db_connection()
            conn.execute('INSERT INTO COMPANY (ID, NAME, AGE, ADDRESS, SALARY ) VALUES (?, ?, ?, ?, ?)',
                         (ID, NAME, AGE, ADDRESS, SALARY ))
            conn.commit()
            conn.close()
            return redirect(url_for('index'))

    return render_template('create.html')


def get_post(post_id):
    conn = get_db_connection()
    post = conn.execute('SELECT * FROM COMPANY WHERE ID = ?',
                        (post_id,)).fetchone()
    conn.close()
    if post is None:
        abort(404)
    return post

@app.route('/<int:id>/edit/', methods=('GET', 'POST'))
def edit(id):
    post = get_post(id)

    if request.method == 'POST':


        NAME = request.form['NAME']
        AGE = request.form['AGE']
        ADDRESS = request.form['ADDRESS']
        SALARY = request.form['SALARY']
        if not NAME:
            flash('Title is required!')
        elif not AGE :
            flash('Content is required!')
        else:
            conn = get_db_connection()
            conn.execute('UPDATE COMPANY SET NAME = ?, AGE = ?, ADDRESS = ?, SALARY = ?'
                         ' WHERE ID = ?',
                         ( NAME, AGE, ADDRESS, SALARY, id))

            conn.commit()
            conn.close()
            return redirect(url_for('index'))

    return render_template('edit.html', post=post)

@app.route('/<int:id>/delete/', methods=('POST',))
def delete(id):
    post = get_post(id)
    conn = get_db_connection()
    conn.execute('DELETE FROM COMPANY WHERE ID = ?', (id,))
    conn.commit()
    conn.close()
    flash('was successfully deleted!')
    return redirect(url_for('index'))


threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

 * ngrok tunnel "https://5f97-34-139-252-42.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
